In [1]:
import pandas as pd
df_spo_wiki=pd.read_csv('spo_wiki.csv')
df_spo_wiki.info()
df_spo_wiki.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    768 non-null    object
 1   predicateLabel  768 non-null    object
 2   objectLabel     768 non-null    object
 3   subject         768 non-null    object
 4   object          768 non-null    object
 5   class           768 non-null    object
dtypes: object(6)
memory usage: 36.1+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object,class
0,pearl,instance of,spherical body,Q43436,Q112511193,Q83437
1,ruby,color,red,Q43088,Q3142,Q83437
2,ruby,described by source,Brockhaus and Efron Encyclopedic Dictionary,Q43088,Q602358,Q83437
3,emerald,streak color,white,Q43513,Q23444,Q83437
4,diamond,streak color,white,Q5283,Q23444,Q83437


In [2]:
df=df_spo_wiki
df_pivot = pd.pivot_table(
    df.reset_index(),
    # columns=['col1'],
    index=['predicateLabel'],
    values=['index'], 
    aggfunc={
        'index': ["count"],
    },
    fill_value=0
)
df_pivot.columns = ['_'.join(col).strip() for col in df_pivot.columns.values]
df_pivot.sort_values('index_count',ascending=False).head(50)

,index_count
predicateLabel,
subclass of,286
described by source,135
instance of,53
different from,34
topic's main category,26
field of this occupation,18
color,18
has use,17
has part(s),16


In [3]:
values_to_drop = {
    'described by source',
    'different from',
    "topic's main category",
    'on focus list of Wikimedia project',
    "topic's main template",
    'maintained by WikiProject'
}
df2 = df[~df['predicateLabel'].isin(values_to_drop)]
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
Index: 558 entries, 0 to 767
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    558 non-null    object
 1   predicateLabel  558 non-null    object
 2   objectLabel     558 non-null    object
 3   subject         558 non-null    object
 4   object          558 non-null    object
 5   class           558 non-null    object
dtypes: object(6)
memory usage: 30.5+ KB


,subjectLabel,predicateLabel,objectLabel,subject,object,class
0,pearl,instance of,spherical body,Q43436,Q112511193,Q83437
1,ruby,color,red,Q43088,Q3142,Q83437
3,emerald,streak color,white,Q43513,Q23444,Q83437
4,diamond,streak color,white,Q5283,Q23444,Q83437
6,emerald,subclass of,material,Q43513,Q214609,Q83437


In [4]:
import json
def spo2kg(df_spo_wiki):
    df = df_spo_wiki
    df_spo = df_spo_wiki[['subjectLabel','predicateLabel','objectLabel']]
    
    metadata = {}
    # Base Wikipedia URL for metadata
    wiki_base_url = "https://en.wikipedia.org/wiki/"
    
    for i in df.itertuples():
        # print(i.subjectLabel)
        
        # Add subject metadata (Wikidata ID and Wikipedia URL)
        if i.subjectLabel not in metadata:
            metadata[i.subjectLabel] = {
                "url": f"{wiki_base_url}{i.subjectLabel.replace(' ', '_')}",
                "wiki_id": i.subject
            }
        
        # Add object metadata (Wikidata ID and Wikipedia URL)
        if i.objectLabel not in metadata:
            metadata[i.objectLabel] = {
                "url": f"{wiki_base_url}{i.objectLabel.replace(' ', '_')}",
                "wiki_id": i.object
            }
    
    return df_spo, metadata

df_spo, metadata = spo2kg(df2)
df_spo.info()
df_spo.head()

<class 'pandas.core.frame.DataFrame'>
Index: 558 entries, 0 to 767
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   subjectLabel    558 non-null    object
 1   predicateLabel  558 non-null    object
 2   objectLabel     558 non-null    object
dtypes: object(3)
memory usage: 17.4+ KB


,subjectLabel,predicateLabel,objectLabel
0,pearl,instance of,spherical body
1,ruby,color,red
3,emerald,streak color,white
4,diamond,streak color,white
6,emerald,subclass of,material


In [5]:
csv_file= 'data_drop.csv'
metadata_file='metadata_drop.json'

df_spo.to_csv(csv_file, header=False, index=False)
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2)

In [6]:
!kg add -f data_drop.csv
!kg meta -f metadata_drop.json
!kg start

🎉 Starting the app.
 * Serving Flask app 'kgsearch.app.app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
^C


In [7]:
df2

,subjectLabel,predicateLabel,objectLabel,subject,object,class
0,pearl,instance of,spherical body,Q43436,Q112511193,Q83437
1,ruby,color,red,Q43088,Q3142,Q83437
3,emerald,streak color,white,Q43513,Q23444,Q83437
4,diamond,streak color,white,Q5283,Q23444,Q83437
6,emerald,subclass of,material,Q43513,Q214609,Q83437
...,...,...,...,...,...,...
762,mezzotinter,subclass of,engraver,Q61155183,Q329439,Q2519376
763,seal cutter,instance of,artistic profession,Q89675973,Q88789639,Q2519376
764,medal engraver,field of this occupation,medal-engraving,Q42296496,Q113921884,Q2519376
765,seal cutter,product or material produced or service provided,seal,Q89675973,Q3850202,Q2519376


In [8]:
df3 = df2[['subject','object']].melt()[['value']].drop_duplicates()
df3.columns = ['wiki_id']
df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
Index: 424 entries, 0 to 1114
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   wiki_id  424 non-null    object
dtypes: object(1)
memory usage: 6.6+ KB


,wiki_id
0,Q43436
1,Q43088
2,Q43513
3,Q5283
23,Q4700957


In [10]:
df3.to_csv('e.csv', index=False)